# Multiprocessing in Windows

## How Windows Differs from Linux

In Windows, processes are created using the `spawn` method rather than the `fork` method used on Linux. This is a fundamental difference with important implications for how you write multiprocessing code.

### The Spawn Method

When Windows spawns a new process, it starts a fresh Python interpreter and re-imports the main module from scratch. This means:

1. **The entire script runs again in each child process** - Unlike Linux where child processes inherit a snapshot of the parent's memory, Windows child processes start fresh and execute the module-level code from the beginning.

2. **Top-level code executes in every process** - Any code that isn't protected will run once in the parent process and again in each child process, potentially causing infinite recursion if you're creating a processes in that top-level code.

3. **Functions and data must be importable** - The target functions and any data structures you pass to child processes need to be defined at module level (not inside `if __name__ == '__main__':` blocks) so they can be imported by the child processes.

### The `if __name__ == '__main__':` Guard

This is why the `if __name__ == '__main__':` guard is essential on Windows. When the main module is imported by a child process, `__name__` will be set to the module name (not `'__main__'`), so code inside the guard won't execute in child processes. This prevents infinite process spawning.

Here's what happens without the guard on Windows:

In [ ]:
# BAD: This will cause infinite process creation on Windows
# If you run this on Windows it will hang. Cancel the execution using Ctrl+C
import multiprocessing

def worker():
    print("Working...")

# This code runs in parent AND every child process
p = multiprocessing.Process(target=worker)
p.start()
p.join()

Working...


Each child process will execute the script again, creating another child process, which creates another child process, and so on until your system runs out of resources.

With the guard, it works correctly:

In [3]:
# GOOD: Safe on all platforms
import multiprocessing

def worker():
    print("Working...")

if __name__ == '__main__':
    # This only runs in the parent process
    p = multiprocessing.Process(target=worker)
    p.start()
    p.join()

Working...


### Writing Portable Code

If you want your multiprocessing code to work on both Linux and Windows:

1. **Always use the `if __name__ == '__main__':` guard** around the code that creates processes
2. **Define functions at module level** (outside the guard) so child processes can import them
3. **Keep data initialization inside the guard** when possible, or be aware that it will be re-executed in Windows child processes

The examples in the main multiprocessing notebook ([03_multiprocessing.ipynb](03_multiprocessing.ipynb)) and elsewhere in this course have been simplified to work in the Linux/Codespaces environment without guards. If you want to run those examples on Windows, you'll need to add the appropriate guards around the process-creation code.